# Lineární klasifikace do více tříd pomocí funkce Softmax

Úloha zaměřená na implementaci paralelního lineární klasifikáru pomocí funkce softmax.


### Softmax
- Funkce softmax má c vstupů a c výstupů. 
- Všechny výstupy jsou kladná čísla. 
- Součet všech výstupů dohromady je roven číslu 1.
$$\widehat{y_c} = softmax(u) = \frac{e^{u_c}}{\sum_{d=0}^{c} {e^{u_d}}} $$

Výsledkem klasifikace je třída, pro kterou je hodnota softmaxu nejvyšší.

### Odhad parametrů modelu

Model nemá jeden vektor parametrů $\theta$, ale matici parametrů $\Theta$

Příklad:
- Vektor $x$ má dimenzi 3 (3 příznaky)
- Klasifikujeme do 4 tříd
$$ \Theta^T = 
        \begin{bmatrix}
        [\theta_{00} & \theta_{01} & \theta_{02} & \theta_{03}] \\
        [\theta_{10} & \theta_{11} & \theta_{12} & \theta_{13}] \\
        [\theta_{20} & \theta_{21} & \theta_{22} & \theta_{23}] \\
        [\theta_{30} & \theta_{31} & \theta_{32} & \theta_{33}] \\
        \end{bmatrix}  $$

#### Pomoc sumy:

$$ \Theta_{t+1} = \Theta_{t} - \alpha \sum_{i=0}^{N} {x_i}^T (\widehat{y_i} - y_i) $$
kde $\alpha$ je velikost kroku (learning rate), 

$y_i$ je vektor, obsahující hodnotu 1 na indexu správné třídy: $y_i \in [...0, 0, 1, 0,..]$,

$\widehat{y_i} = softmax(x_i^T \Theta)^T$


#### Pomocí  matice:
Maticová implementace je řádově výpočetně efektivnější.

$$ \Theta_{t+1} = \Theta_{t} - \alpha X^T (\widehat{Y} - Y) $$
$Y$ je matice nul a jedniček dle indexů tříd v datech (one_hot_encoding), 

#### Postup:
1. Parametr $\theta$ (resp. váhy $w$ a bias $b$) se inicializuje na malé náhodné hodnoty
2. Na vstup se přivedou všechna trénovací data a na výstupu se tím pádem objeví posloupnost nul a jedniček (obsahuje chyby)
3. Na základě chyb se upraví hodnoty vah
4. Kroky 2. a 3. se postupně opakují dokud dostatečně klesá chybovost klasifikátoru (nebo podle počtu iteraci)



### Klasifikace pro 2 třídy:


In [ ]:
import numpy as np
import usu
import matplotlib.pyplot as plt

def show_loss(iterations, loss, epoch=False):
    plt.plot(iterations, loss)
    plt.xlabel("Epocha" if epoch else "Iterace")
    plt.ylabel("Loss")
    plt.title("Průběh trénování")
    plt.show()

npzfile = np.load('data/data_07_2cl.npz')

data = npzfile['data']
ref = npzfile['ref']
data.shape, ref.shape



In [ ]:
usu.drawSoftmax(data,ref)


In [ ]:
def softmax(u):
    """
    vstupem muze byt skalar, vektor, nebo matice 
    """
    #################################################################
    # ZDE DOPLNIT

    # Numerical stability: subtract max value to prevent overflow
    u_shifted = u - np.max(u, axis=0, keepdims=True)
    exp_u = np.exp(u_shifted)
    y = exp_u / np.sum(exp_u, axis=0, keepdims=True)
    
    #################################################################
    return y

usu.checkSoftmax(softmax)

In [ ]:
def computeThetaGD(x, y, alpha=0.01, iterations=1000, plot_loss=False):
    # pro kazdou iteraci vypoctete loss a zapiste ho na odpovidajici index v promenne
    loss = np.zeros(iterations)

    #################################################################
    # ZDE DOPLNIT

    # Flatten y if needed (handle both (N,) and (N,1) shapes)
    y = y.flatten()
    
    # Get number of samples and features
    n_samples = x.shape[0]
    n_features = x.shape[1]
    n_classes = np.max(y) + 1
    
    # Add bias term to x
    X = np.c_[np.ones(n_samples), x]
    
    # Initialize theta with small random values
    theta = np.random.randn(n_features + 1, n_classes) * 0.01
    
    # One-hot encode the target classes
    Y = np.zeros((n_samples, n_classes))
    Y[np.arange(n_samples), y] = 1
    
    # Gradient descent iterations
    for iteration in range(iterations):
        # Compute predictions using softmax
        u = X @ theta  # Linear combination
        Y_pred = softmax(u.T).T  # Apply softmax row-wise
        
        # Compute gradient
        gradient = X.T @ (Y_pred - Y)
        
        # Update theta
        theta = theta - alpha * gradient
        
        # Compute loss (cross-entropy)
        loss[iteration] = -np.sum(Y * np.log(Y_pred + 1e-15)) / n_samples
    
    #################################################################

    if plot_loss:
        show_loss(np.arange(iterations), loss)

    return theta


In [ ]:
theta = computeThetaGD(data, ref, plot_loss=True)
theta

In [ ]:
usu.drawSoftmax(data, ref, theta, softmax)


In [ ]:
def predict(x,theta):
    #################################################################
    # ZDE DOPLNIT
    # Add bias term to x
    X = np.c_[np.ones(x.shape[0]), x]
    
    # Compute linear combination
    u = X @ theta
    
    # Apply softmax and get predicted classes
    y_prob = softmax(u.T).T
    classes = np.argmax(y_prob, axis=1)
    #################################################################
    return classes


In [ ]:
x_pred = np.array([[1, 1],[ 2, 2]])
predict(x_pred,theta)


### Klasifikace pro více tříd:
#### 3 třídy:


In [ ]:
npzfile = np.load('data/data_07_3cl_ez.npz') 

data = npzfile['data']
ref = npzfile['ref']

In [ ]:
theta = computeThetaGD(data, ref, plot_loss=True)
theta

In [ ]:
usu.drawSoftmax(data, ref, theta, softmax)


In [ ]:
x_pred = np.array([[-3, 1],[ 0, -2],[ 3, 2]])
predict(x_pred,theta)


#### 4 třídy:

In [ ]:
npzfile = np.load('data/data_07_4cl.npz')

data = npzfile['data']
ref = npzfile['ref']


In [ ]:
theta = computeThetaGD(data, ref, plot_loss=True)
theta

In [ ]:
usu.drawSoftmax(data, ref, theta, softmax)


In [ ]:
x_pred = np.array([[-3, -1],[ -3, 2],[ 3, -1],[ 3, 2]])
predict(x_pred,theta)


#### 5 tříd:

In [ ]:
npzfile = np.load('data/data_07_5cl.npz')

data = npzfile['data']
ref = npzfile['ref']


In [ ]:
theta = computeThetaGD(data, ref, plot_loss=True)
theta

In [ ]:
usu.drawSoftmax(data, ref, theta, softmax)


In [ ]:
x_pred = np.array([[-3, -1],[ -3, 2],[ 3, -1],[ 3, 2],[0, 0]])
predict(x_pred,theta)


# BONUS: Rozpoznávání číslovek (MNIST database) a mini-batch gradient descent

### **Dosáhněte accuracy (vypočtené na konci) alespoň 91 %.**

In [ ]:
npzfile = np.load('data/data_07_mnist_train.npz') 

data = npzfile['data']
ref = npzfile['ref']

Prohlédněte si prvních 20 trénovacích vzorků:

In [ ]:
plt.figure(figsize=(10, 6))
for i in range(20):
    # Prevedeni linearizovych dat zpet na 2D matici.
    img = np.reshape(data[i], (28, 28))
    plt.subplot(4, 5, i+1)
    plt.imshow(img, "gray_r")
    plt.title(ref[i])
    plt.axis("off") 
plt.show()

Naimplementuje trénování modelu pomocí mini-batch gradient descent:

Mini-batch gradient descent vzhledem k množství dat zefektivní trénování (model se natrénuje rychleji a lépe zobecňuje).

Vyjděte z předchozí implementace `computeThetaGD` a upravte ji.

Počet iterací (u GD ekvivalentní průchodu všemi testovacími daty) bude nahrazen počtem tzv. epoch.

Jedna epocha probíhá následovně:
- Na začátku se náhodně zamíchají testovací data.
- Všechna testovací data se projdou po mini-batchích, jejichž délka je vždy rovna zadané `batch_size`. Na základě každého mini-batche se aktualizují váhy (u `computeThetaGD` se aktualizovaly váhy na základě všech dat). Pokud by na konci epochy zbyly vzorky, jejichž počet už je menší než `batch_size`, tak se přeskočí.
- Na konci epochy se spočítá loss na základě všech dat pro vizualizaci.

TIP: Vhodně normalizujte úpravu vah, aby její výsledná velikost nezávisela na `batch_size` a parametry `alpha` a `batch_size` jste mohli ladit nezávisle na sobě.

In [ ]:
def computeThetaSGD(x, y, *, alpha, epochs, batch_size, plot_loss=False):
    # zafixovani nahody pri trenovani
    np.random.seed(0)

    # na konci kazde epochy vypoctete loss (ze vsech trenovacich vzorku) a zapiste ho na odpovidajici index v promenne
    loss = np.zeros(epochs)

    #################################################################
    # ZDE DOPLNIT

    # Flatten y if needed (handle both (N,) and (N,1) shapes)
    y = y.flatten()
    
    # Get number of samples, features, and classes
    n_samples = x.shape[0]
    n_features = x.shape[1]
    n_classes = np.max(y) + 1
    
    # Add bias term to x
    X = np.c_[np.ones(n_samples), x]
    
    # Initialize theta with small random values
    theta = np.random.randn(n_features + 1, n_classes) * 0.01
    
    # One-hot encode the target classes
    Y = np.zeros((n_samples, n_classes))
    Y[np.arange(n_samples), y] = 1
    
    # Mini-batch gradient descent
    for epoch in range(epochs):
        # Shuffle data at the beginning of each epoch
        indices = np.random.permutation(n_samples)
        X_shuffled = X[indices]
        Y_shuffled = Y[indices]
        
        # Process mini-batches
        for i in range(0, n_samples - batch_size + 1, batch_size):
            # Get mini-batch
            X_batch = X_shuffled[i:i+batch_size]
            Y_batch = Y_shuffled[i:i+batch_size]
            
            # Compute predictions using softmax
            u = X_batch @ theta
            Y_pred = softmax(u.T).T
            
            # Compute gradient and normalize by batch_size
            gradient = X_batch.T @ (Y_pred - Y_batch) / batch_size
            
            # Update theta
            theta = theta - alpha * gradient
        
        # Compute loss on all training data at the end of epoch
        u_all = X @ theta
        Y_pred_all = softmax(u_all.T).T
        loss[epoch] = -np.sum(Y * np.log(Y_pred_all + 1e-15)) / n_samples
    
    #################################################################

    if plot_loss:
        show_loss(np.arange(epochs), loss, epoch=True)

    return theta


In [ ]:
 # ZDE DOPLNIT hodnoty hyperparametrů (dostatečný model lze natrénovat během 5-10 sekund)
theta = computeThetaSGD(data, ref, batch_size=128, epochs=50, alpha=0.1, plot_loss=True)
#################################################################

theta

Prohlédněte si natrénované váhy (s výjimkou biasu):

In [ ]:
for i in range(theta.shape[1]):
    # Prvni vaha je bias, zbytek odpovida bodum v obrazu, takze je preskupime na 2D obraz.
    img = np.reshape(theta.T[i][1:], (28, 28))
    plt.subplot(2, 5, i+1)
    plt.imshow(img, "gray_r")
    plt.title(str(i))
    plt.axis("off") 
plt.show()

In [ ]:
npzfile = np.load('data/data_07_mnist_test.npz') 

test_data = npzfile['data']
test_ref = npzfile['ref']

Otestujte prvních 20 testovacích vzorků:

In [ ]:
n = 20
x = test_data[:n, :]
y = test_ref[:n]

#################################################################
# ZDE DOPLNIT - Zjistete predikovane tridy pro `x`

y_pred = predict(x, theta)

#################################################################

plt.figure(figsize=(10, 6))
for i in range(n):
    # Prevedeni linearizovych dat zpet na 2D matici.
    img = np.reshape(x[i, :], (28, 28))
    plt.subplot(4, 5, i+1)
    plt.tight_layout(pad=3.0)
    plt.imshow(img, "gray_r")
    plt.title(f"predicted: {y_pred[i]}\nactual: {y[i]}")
    plt.axis("off")
plt.show()

Accuracy – přesnost natrénovaného modelu:

In [ ]:
#################################################################
# ZDE DOPLNIT - Zjistete predikovane tridy pro vsechna testovaci data a procento spravne predikovanych dat (accuracy).

test_pred = predict(test_data, theta)
accuracy = np.mean(test_pred == test_ref)

#################################################################

print(f"Accuracy: {accuracy * 100}%")

Navíc: všimněte si, jak se mění grafy vah, když zvyšujete/snižujete batch size.